In [50]:
#https://www.kaggle.com/datasets/hyunkic/twitter-depression-dataset
#https://github.com/eddieir/Depression_detection_using_Twitter_post/blob/master/depression.py
import pandas as pd

df = pd.read_csv("/content/depressive_tweets_processed.csv",sep = '|', header = None, usecols = range(0,9))
df.columns = [3, "date", "time", "time-zone", "user", "tweet", 0, 1, 2]
df = df.drop([0, 1, 2, 3, "time-zone", "time", "date", "user"], axis = 1)
df['label'] = [1] * len(df['tweet']) # 1 cause these are all depressed tweets


In [51]:
df2_d = pd.read_csv("/content/d_tweets.csv")
df2_d = df2_d[["tweet"]]
df2_d['label'] = [1] * len(df2_d['tweet'])

df2_nd = pd.read_csv("/content/non_d_tweets.csv")
df2_nd = df2_nd[["tweet"]]
df2_nd['label'] = [0] * len(df2_nd['tweet'])
print(len(df2_nd['label']))
df = pd.concat([df2_d, df2_nd, df])
df.reset_index(inplace = True)


4809


In [52]:
df['label'].value_counts()


label
1    5841
0    4809
Name: count, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10650 entries, 0 to 10649
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   10650 non-null  int64 
 1   tweet   10618 non-null  object
 2   label   10650 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 249.7+ KB


In [53]:
import re
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download(['wordnet', 'stopwords', 'punkt'])
df['tweet'] = df['tweet'].astype(str)
stop=set(stopwords.words('english'))
filter = set(['iyanlavanzant', 'fixmylife', 'iyanla', 'http'])
l = WordNetLemmatizer()
def process(text):
  text=re.sub(r'\s+',' ',text, flags=re.I)
  #remove spl characters other than A-Z,a-z,0-9 or _
  text=re.sub(r'\W',' ',text)

  #remove single character
  text=re.sub(r'\s+[a-zA-Z]\s+',' ',text)

  #remove character not alphabetical
  text=re.sub(r'[^a-zA-Z\s]','',text)

  text=text.lower()

  words= word_tokenize(text)

  words=[l.lemmatize(i) for i in words]
  Words=[word for word in words if word not in stop and word not in filter]


  Words=[word for word in Words if len(word)>2]

  #removing duplicate words
  indices = np.unique(Words, return_index=True)[1]
  cleaned_text = np.array(Words)[np.sort(indices)].tolist()

  return cleaned_text

df['tweet'] = [process(i) for i in df['tweet']]
x = df['tweet']
y = df['label']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = .2, random_state=42)

import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
max_vocab = 20000
t = Tokenizer(num_words=max_vocab)
t.fit_on_texts(x_train)
word_idx = t.word_index
v = len(word_idx)
print('vocab size', v)
x_train = t.texts_to_sequences(x_train)
x_test = t.texts_to_sequences(x_test)

maxlen = 100
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
x_train

x_test

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


vocab size 14387


array([[   0,    0,    0, ...,  730,  326,  126],
       [   0,    0,    0, ...,  606,  366,    4],
       [   0,    0,    0, ...,  495,  659,  353],
       ...,
       [   0,    0,    0, ...,  162,  164,  187],
       [   0,    0,    0, ...,  278, 1465,  302],
       [   0,    0,    0, ...,    0,    0,    1]], dtype=int32)